In [56]:
%pwd

'c:\\Users'

In [57]:
import os

In [58]:
os.chdir("../")

In [59]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [60]:
def load_pdf_file(data):
    loader =DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [61]:
extracted_data=load_pdf_file(data='Data/')

FileNotFoundError: Directory not found: 'Data/'

In [ ]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter (chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents (extracted_data)
    return text_chunks

In [ ]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 2345


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [ ]:
embeddings=download_hugging_face_embeddings()

C:\Users\Afrith\AppData\Local\Temp\ipykernel_2260\4043855499.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [ ]:
query_result=embeddings.embed_query("hello world")
print("Length",len(query_result))

Length 384


In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [ ]:
pip uninstall -y pinecone pinecone-client pinecone-plugin-inference pinecone-grpc


Found existing installation: pinecone 6.0.1
Uninstalling pinecone-6.0.1:
  Successfully uninstalled pinecone-6.0.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade pinecone


  Using cached pinecone-6.0.1-py3-none-any.whl.metadata (8.8 kB)
Using cached pinecone-6.0.1-py3-none-any.whl (421 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install --upgrade jupyter ipywidgets


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [62]:
index_name="medicalbot"
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [63]:
import os
os.environ["PINECONE_API_KEY"]= PINECONE_API_KEY

In [64]:
index_name="medicalbot"
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_existing_index(
    
    index_name=index_name,
    embedding=embeddings,
)

In [65]:
docsearch

In [66]:
retriever=docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [67]:
retrived_docs=retriever.invoke("what is Acne?")

In [68]:
retrived_docs

[Document(id='a61b358a-3686-4b44-8685-193d5649fc67', metadata={'author': 'Tierney, Lawrence M.; Saint, Sanjay.; Whooley, Mary A.', 'creationdate': '2010-12-22T13:15:56+00:00', 'creator': 'PyPDF', 'keywords': '0071637907\r\n9780071637909', 'moddate': '2018-08-02T09:28:12+05:30', 'page': 412.0, 'page_label': '400', 'producer': 'PyPDF', 'source': 'Data\\Current Essentials of Medicine.pdf', 'subject': 'McGraw-Hill Medical', 'title': 'Current Essentials of Medicine', 'total_pages': 607.0}, page_content='400 Current Essentials of Medicine\nAcne Vulgaris\n■ Essentials of Diagnosis\n• Often occurs at puberty, though onset may be delayed until the\nthird or fourth decade\n• Open and closed comedones the hallmarks\n• Severity varies from comedone to papular or pustular inﬂamma-\ntory acne to cysts or nodules\n• Face, neck, upper chest, and back may be affected\n• Pigmentary changes and severe scarring can occur\n■ Differential Diagnosis\n• Acne rosacea, perioral dermatitis, gram-negative follicu

In [69]:
import openai

client = openai.OpenAI(
    api_key="77d82fc4e93b73b5d222772f81c48afba97e8f1b0d2bf8bfd5e8835f3df3abc9",  
    base_url="https://api.together.xyz"
)

completion = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo-128K",  
    messages=[{"role": "user", "content": "Explain recursion in Python."}])

In [70]:
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate


In [ ]:
llm = ChatOpenAI(
    openai_api_key="77d82fc4e93b73b5d222772f81c48afba97e8f1b0d2bf8bfd5e8835f3df3abc9",
    openai_api_base="https://api.together.xyz",
    model_name="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo-128K"
)

system_prompt=(
    "You are an assistant fro question-answering tasks."
    "use the following pieces of retrived context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know. use three sentences maximum anmd keep the answer concise"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])


retrieval_chain = create_stuff_documents_chain(llm, prompt)
rag_chain=create_retrieval_chain(retriever,retrieval_chain)




In [74]:
response = rag_chain.invoke({"input": "what is marketing?."})
print(response["answer"])

I don't know the answer.
